# 05 · Evaluación Final — AVSI
**Artificial Vision Stacking Inspection** · *2025-10-22*

Consolida resultados de entrenamiento y prueba, y genera gráficas finales para el informe:
- Carga métricas de `03_modelado` (history, matriz de confusión, reporte).
- Integra resultados de `04_optimizacion` (mejor configuración).
- Produce un **resumen ejecutivo** con KPIs clave.


## 1. Rutas y archivos esperados

In [ ]:

from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path('.').resolve()
RESULTS_DIR = ROOT / 'results' / 'metrics'
FIG_DIR = ROOT / 'results' / 'figures'
MODELS_DIR = ROOT / 'models'

files = {
    'history': RESULTS_DIR / 'history.csv',
    'cm': RESULTS_DIR / 'confusion_matrix.csv',
    'report': RESULTS_DIR / 'classification_report.csv',
    'summary': RESULTS_DIR / 'summary.json',
    'grid': RESULTS_DIR / 'grid_results.csv'
}
files


## 2. Carga de resultados y visualizaciones

In [ ]:

def safe_read_csv(path):
    try:
        return pd.read_csv(path)
    except Exception as e:
        print('[Aviso]', e)
        return pd.DataFrame()

hist = safe_read_csv(files['history'])
cm = safe_read_csv(files['cm'])
report = safe_read_csv(files['report'])
grid = safe_read_csv(files['grid'])

# history
if not hist.empty:
    plt.figure()
    plt.plot(hist['train_loss'], label='train_loss')
    plt.plot(hist['val_loss'], label='val_loss')
    plt.title('Pérdida — entrenamiento')
    plt.xlabel('Época'); plt.ylabel('Loss'); plt.legend()
    plt.savefig(FIG_DIR / 'final_loss.png', bbox_inches='tight')
    plt.show()

    plt.figure()
    plt.plot(hist['train_acc'], label='train_acc')
    plt.plot(hist['val_acc'], label='val_acc')
    plt.title('Exactitud — entrenamiento')
    plt.xlabel('Época'); plt.ylabel('Accuracy'); plt.legend()
    plt.savefig(FIG_DIR / 'final_accuracy.png', bbox_inches='tight')
    plt.show()
else:
    print('[Aviso] No se encontró history.csv (ejecuta 03_modelado).')

# matriz de confusión
if not cm.empty:
    plt.figure()
    plt.imshow(cm.values, interpolation='nearest')
    plt.title('Matriz de confusión (test)')
    plt.colorbar()
    plt.xlabel('Predicción'); plt.ylabel('Real')
    plt.savefig(FIG_DIR / 'final_confusion_matrix.png', bbox_inches='tight')
    plt.show()
else:
    print('[Aviso] No se encontró confusion_matrix.csv.')

# reporte clasificación (opcional)
if not report.empty:
    display(report.head())
else:
    print('[Aviso] No se encontró classification_report.csv.')

# mejores configs de grid
if not grid.empty:
    display(grid.head())
else:
    print('[Aviso] No se encontró grid_results.csv (ejecuta 04_optimizacion).')


## 3. Resumen ejecutivo (KPIs)

In [ ]:

summary = {}
if files['summary'].exists():
    with open(files['summary'], 'r') as f:
        summary = json.load(f)

kpis = {
    'best_val_acc': summary.get('best_val_acc', None),
    'test_accuracy': summary.get('test_accuracy', None),
    'num_classes': len(summary.get('classes', [])) if summary.get('classes') else None,
}
kpis


## 4. Comparativa vs. baseline (opcional)

In [ ]:

# Si dispones de un CSV 'baseline.csv' con columnas: metric,value
BASELINE = RESULTS_DIR / 'baseline.csv'
if BASELINE.exists():
    base = pd.read_csv(BASELINE)
    print('Baseline:'); display(base)
else:
    print('[Info] No hay baseline.csv — puedes crearlo con tus métricas iniciales (dataset_100).')


## 5. Exportar reporte simple

In [ ]:

report_txt = RESULTS_DIR / 'final_report.txt'
lines = []
lines.append('AVSI — Evaluación Final\n')
lines.append('=======================\n\n')
lines.append('KPIs:\n')
for k, v in kpis.items():
    lines.append(f"- {k}: {v}\n")
lines.append('\nArchivos generados en results/figures y results/metrics.\n')

with open(report_txt, 'w', encoding='utf-8') as f:
    f.writelines(lines)

str(report_txt)


## 6. Notas
- Para métricas de **detección** (mAP/IoU), integra un detector (por ejemplo, Faster R-CNN/YOLO) y calcula métricas específicas.
- Este notebook consolida **clasificación**; ajusta según tu setup final.
